There is are no errors in the forecasts for weather conditions in citylearn. THis notbook calculates the mean error introduced by the ACG attack.

In [24]:
import numpy as np
import pandas as pd

The observations contain current and forecast measurements, we will analyze how the attack affects the accuracy of forecasts

Only keep weather/pricing data

In [25]:
obs_spread = pd.read_csv('citylearn_challenge_2022_phase_2 spread.csv',
                            index_col=0)

In [26]:
def obs_pred_error(df_obs, agg_func_list:list=['mean','sem','std','min','max']):
    """For a df of CityLearn Observations, calculates the prediction error for prediction features
    agg_func_list is a list of panadas methods with can be used by the .agg(method) and applied to the results."""
    #only keep feature with associated predictions
    df_obs = df_obs[df_obs.columns[df_obs.columns.str.contains(f'temp|humidity|solar|pricing')]]
    #realign data do that forecasts are in the same row as the measurements the predict
    forecasts = [6,12,24] #hours ahead predicted
    for i in forecasts:
        df_obs[df_obs.columns[df_obs.columns.str.contains(f'{i}')]] =\
            df_obs.loc[:,df_obs.columns[df_obs.columns.str.contains(f'{i}')]].shift(i)
    #calculate the difference between the measurements and forecasts in each row
    for measurement in df_obs.loc[:,~df_obs.columns.str.contains('predicted')].columns: #column names which don't contain predicted
        df_obs[df_obs.columns[df_obs.columns.str.contains(f'{measurement}_predicted')]] = \
            df_obs[df_obs.columns[df_obs.columns.str.contains(f'{measurement}_predicted')]].sub(df_obs[measurement], axis='rows').abs() #difference between the measurement and prediction
    #rename predictions
    df_obs.rename(columns=lambda name: name + '_error' if 'predicted' in name else name, inplace=True)
    #return mean errors
    return df_obs[df_obs.columns[df_obs.columns.str.contains('error')]].agg(agg_func_list).transpose()

In [27]:
error = obs_pred_error(pd.read_csv(r'default SAC 500 norm space results\toggle_binary_myPGD_03_mask_time_and_solar_scale_consumption_eps_clipped_adv_obs-a.csv',
                index_col=0,
                usecols = lambda x: x != 'a', #excludes a col
                ))

c:\Users\kbrod\anaconda3\envs\CityLearnART\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Normalized

In [33]:
error

,mean,sem,std,min,max
outdoor_dry_bulb_temperature_predicted_6h_error,0.028718,0.000267,0.025002,0.0,0.06
outdoor_dry_bulb_temperature_predicted_12h_error,0.028331,0.000268,0.025033,0.0,0.06
outdoor_dry_bulb_temperature_predicted_24h_error,0.027957,0.000268,0.025040,0.0,0.06
outdoor_relative_humidity_predicted_6h_error,0.028968,0.000271,0.025322,0.0,0.06
outdoor_relative_humidity_predicted_12h_error,0.029538,0.000270,0.025220,0.0,0.06
outdoor_relative_humidity_predicted_24h_error,0.028679,0.000270,0.025235,0.0,0.06
diffuse_solar_irradiance_predicted_6h_error,0.019262,0.000225,0.021017,0.0,0.06
diffuse_solar_irradiance_predicted_12h_error,0.023167,0.000240,0.022493,0.0,0.06
diffuse_solar_irradiance_predicted_24h_error,0.016320,0.000215,0.020089,0.0,0.06
direct_solar_irradiance_predicted_6h_error,0.019912,0.000216,0.020249,0.0,0.06


With units

In [39]:
error*obs_spread.loc[obs_spread.index.str.contains(f'predicted')].values

,mean,sem,std,min,max
outdoor_dry_bulb_temperature_predicted_6h_error,0.763899,0.007108,0.665053,0.0,1.596001
outdoor_dry_bulb_temperature_predicted_12h_error,0.753613,0.007120,0.665890,0.0,1.596000
outdoor_dry_bulb_temperature_predicted_24h_error,0.743659,0.007127,0.666074,0.0,1.596000
outdoor_relative_humidity_predicted_6h_error,2.607103,0.024359,2.278978,0.0,5.400002
outdoor_relative_humidity_predicted_12h_error,2.658403,0.024270,2.269833,0.0,5.400002
outdoor_relative_humidity_predicted_24h_error,2.581139,0.024301,2.271166,0.0,5.400002
diffuse_solar_irradiance_predicted_6h_error,19.589542,0.228463,21.374383,0.0,61.020020
diffuse_solar_irradiance_predicted_12h_error,23.560781,0.244586,22.875030,0.0,61.020020
diffuse_solar_irradiance_predicted_24h_error,16.597399,0.218594,20.430024,0.0,61.020020
direct_solar_irradiance_predicted_6h_error,18.975717,0.206266,19.297736,0.0,57.180029


Mean of the diffuse means:

In [2]:
(19.589542 + 23.560781 + 16.597399)/3

19.915907333333333